In [34]:
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn import preprocessing
from sklearn import linear_model 
from sklearn import metrics 
from  sklearn.ensemble import AdaBoostClassifier
from  sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
import numpy as np

In [10]:
df = pd.read_csv('data/AirPass.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head(3)

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied


In [11]:
df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median(), inplace=True)
df['Arrival Delay in Minutes'].mean()

15.133392362180475

In [15]:
df.groupby('Gender')['satisfaction'].value_counts()*100/df.shape[0]

Gender  satisfaction           
Female  neutral or dissatisfied    29.058554
        satisfied                  21.687327
Male    neutral or dissatisfied    27.608177
        satisfied                  21.645942
Name: count, dtype: float64

In [16]:
df.groupby('Type of Travel')['satisfaction'].value_counts()*100/df.shape[0]

Type of Travel   satisfaction           
Business travel  satisfied                  40.177472
                 neutral or dissatisfied    28.785225
Personal Travel  neutral or dissatisfied    27.881506
                 satisfied                   3.155798
Name: count, dtype: float64

In [17]:
df.groupby('Class')['satisfaction'].value_counts()*100/df.shape[0]

Class     satisfaction           
Business  satisfied                  33.184478
          neutral or dissatisfied    14.614452
Eco       neutral or dissatisfied    36.614567
          satisfied                   8.374076
Eco Plus  neutral or dissatisfied     5.437712
          satisfied                   1.774715
Name: count, dtype: float64

In [18]:
df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

In [19]:
df=pd.get_dummies(df)
df.shape

(103904, 26)

In [21]:
X = df.drop('satisfaction', axis=1)
y = df.satisfaction

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=26)
y_test.shape

(20781,)

In [25]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 
X_test[0][0]

0.9408251379303

In [28]:
model_lr = linear_model.LogisticRegression()
model_lr.fit(X_train, y_train)
preds_test = model_lr.predict(X_test)
metrics.f1_score(preds_test, y_test)

0.8546883773161146

In [31]:
model_ada = AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=26),random_state=26,learning_rate=0.01)

model_ada.fit(X_train, y_train)
preds_test = model_ada.predict(X_test)
metrics.f1_score(preds_test, y_test)

0.9404794558121674

In [35]:
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
model_for_gs = GradientBoostingClassifier()
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
gs = GridSearchCV(model_for_gs, 
                  params, 
                  cv=3, 
                  scoring=metrics.make_scorer(metrics.f1_score),
                  verbose=5)
 
gs.fit(X_train, y_train)
 
print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END .learning_rate=1.0, n_estimators=1;, score=0.873 total time=   0.1s
[CV 2/3] END .learning_rate=1.0, n_estimators=1;, score=0.870 total time=   0.1s
[CV 3/3] END .learning_rate=1.0, n_estimators=1;, score=0.871 total time=   0.1s
[CV 1/3] END .learning_rate=1.0, n_estimators=2;, score=0.880 total time=   0.2s
[CV 2/3] END .learning_rate=1.0, n_estimators=2;, score=0.878 total time=   0.2s
[CV 3/3] END .learning_rate=1.0, n_estimators=2;, score=0.875 total time=   0.2s
[CV 1/3] END .learning_rate=1.0, n_estimators=4;, score=0.901 total time=   0.4s
[CV 2/3] END .learning_rate=1.0, n_estimators=4;, score=0.896 total time=   0.4s
[CV 3/3] END .learning_rate=1.0, n_estimators=4;, score=0.897 total time=   0.4s
[CV 1/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   0.9s
[CV 2/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   0.9s
[CV 3/3] END .learning_rate=1.0, n_estimators=8;

In [36]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(random_state=26)
model_xgb.fit(X_train,y_train)
preds_test = model_xgb.predict(X_test)
metrics.f1_score(preds_test, y_test)

0.9579785161685312

In [37]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(random_state=26)
model.fit(X_train, y_train)
preds_class = model.predict(X_test)
metrics.f1_score(preds_class, y_test)

Learning rate set to 0.068023
0:	learn: 0.6018089	total: 169ms	remaining: 2m 48s
1:	learn: 0.5020769	total: 186ms	remaining: 1m 32s
2:	learn: 0.4472481	total: 204ms	remaining: 1m 7s
3:	learn: 0.4028675	total: 222ms	remaining: 55.3s
4:	learn: 0.3674724	total: 239ms	remaining: 47.7s
5:	learn: 0.3397844	total: 258ms	remaining: 42.8s
6:	learn: 0.3121211	total: 278ms	remaining: 39.4s
7:	learn: 0.2917499	total: 295ms	remaining: 36.6s
8:	learn: 0.2749039	total: 319ms	remaining: 35.1s
9:	learn: 0.2575191	total: 342ms	remaining: 33.9s
10:	learn: 0.2473690	total: 367ms	remaining: 33s
11:	learn: 0.2377531	total: 390ms	remaining: 32.1s
12:	learn: 0.2279309	total: 420ms	remaining: 31.9s
13:	learn: 0.2212512	total: 450ms	remaining: 31.7s
14:	learn: 0.2100359	total: 493ms	remaining: 32.4s
15:	learn: 0.2025733	total: 532ms	remaining: 32.7s
16:	learn: 0.1942303	total: 552ms	remaining: 31.9s
17:	learn: 0.1877939	total: 579ms	remaining: 31.6s
18:	learn: 0.1832381	total: 598ms	remaining: 30.9s
19:	learn: 

0.960182404626849

In [41]:
from catboost import Pool
from catboost.utils import get_confusion_matrix
cm = get_confusion_matrix(model, Pool(X_train, y_train))
print(cm*100/X_train.shape[0])

[[56.13488445  0.65565487]
 [ 1.53266846 41.67679222]]


In [42]:
pd.DataFrame(
    {
        "feature_importance": model.get_feature_importance(),
        "feature_names": df.drop(columns="satisfaction").columns,
    }
).sort_values(by=["feature_importance"], ascending=False)

,feature_importance,feature_names
6,25.364737,Inflight wifi service
4,18.391876,Type of Travel
11,7.401483,Online boarding
2,7.240564,Customer Type
22,5.420057,Class_Business
17,3.925791,Checkin service
3,3.742350,Age
16,3.640798,Baggage handling
9,3.196286,Gate location
12,3.012455,Seat comfort
